# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = '5.AK93.AKJT72.T4'

# the auction goes:
auction = ["PAD_START","PASS", "PASS", "2D"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 1, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

# what's your bid?

[{'call': 'X', 'insta_score': 0.285, 'expected_score': 8, 'adjustment': 14}, {'call': '3D', 'insta_score': 0.566, 'expected_score': -86, 'adjustment': 28}]


In [3]:
bid.samples

['x.AK9x.AKJTxx.Tx QJ8x.QTxx.x.QJ8x 9xxxx.J8xx..AKxx AKT.x.Q98xxx.9xx 0.97998 \n P-P-2D-3D-P-3S-P-4D-P-4S-P-P-P (7.5)  \n P-P-2D-X-P-2S-P-P-P (7.5) ',
 'x.AK9x.AKJTxx.Tx Q8xx.Qxxx.x.AJxx KJ9xx.Txx..K9xxx ATx.J8.Q98xxx.Q8 0.97852 \n P-P-2D-3D-P-3S-P-4D-P-4S-P-P-P (8.0)  \n P-P-2D-X-P-2S-P-P-P (8.0) ',
 'x.AK9x.AKJTxx.Tx AQxxx.JTxxx.x.Jx JT98x.Qx..KQ98xx Kx.8x.Q98xxx.Axx 0.97824 \n P-P-2D-3D-P-3S-P-4D-P-4S-P-P-P (7.9)  \n P-P-2D-X-P-2S-P-P-P (7.9) ',
 'x.AK9x.AKJTxx.Tx QJxxx.JT.9.QJxxx AT8xx.Q8xxx..Kxx K9.xx.Q8xxxx.A98 0.97550 \n P-P-2D-3D-P-3S-P-4D-P-4S-P-P-P (7.9)  \n P-P-2D-X-P-2H-P-3H-P-4H-P-P-P (11.1) ',
 'x.AK9x.AKJTxx.Tx JTxx.8xx..AK9xxx KQxxx.QJTx.9.J8x A98.xx.Q8xxxx.Qx 0.97424 \n P-P-2D-3D-P-3S-P-4D-P-4S-P-P-P (6.7)  \n P-P-2D-X-P-3D-P-P-P (7.9) ',
 'x.AK9x.AKJTxx.Tx QJTxx.xxx..KQJ9x Kxxx.JTxx.9.Axxx A98.Q8.Q8xxxx.8x 0.97357 \n P-P-2D-3D-P-3S-P-4D-P-P-P (8.3)  \n P-P-2D-X-P-3D-P-P-P (8.9) ',
 'x.AK9x.AKJTxx.Tx KTxxx.T8xx..KQJx AJ98x.Qxx.x.98xx Qx.Jx.Q98xxx.Axx 0.97297 \n P-P-2D-

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [5]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

x.AK9x.AKJTxx.Tx AQ9xx.Qx.x.J8xxx K.Txxx.Q9xx.AKQx JT8xxx.J8x.8x.9x
x.AK9x.AKJTxx.Tx JT9xx.T8xx.x.98x AK8.Qxx.Qxx.AKJx Qxxx.Jx.98x.Qxxx
x.AK9x.AKJTxx.Tx JTxx.T8xx.8.Q8xx AKx.QJx.Qxx.AKJx Q98xx.xx.9xx.9xx
x.AK9x.AKJTxx.Tx 9xx.QJT8.98x.8xx QTxx..xxx.AKQJ9x AKJ8x.xxxxx.Q.xx
x.AK9x.AKJTxx.Tx A8x.QJxxxx.Q8x.x QJTx..xx.AKQJxxx K9xxx.T8x.9x.98x
x.AK9x.AKJTxx.Tx KJTx.8xx.8x.9xxx AQx.Jxx.Qxx.AKQ8 98xxx.QTx.9x.Jxx
x.AK9x.AKJTxx.Tx Q8xx.8x.Q9x.Q8xx AKT.QJx.xxx.AKJx J9xxx.Txxx.8.9xx
x.AK9x.AKJTxx.Tx Qxx.QT8xx.9x.Qxx K9xx..Q8xx.AKJxx AJT8x.Jxxx.x.98x
x.AK9x.AKJTxx.Tx 98xx.QJxxx.xxx.x KJTx.T.8.AKQ9xxx AQxx.8xx.Q9x.J8x
x.AK9x.AKJTxx.Tx KQxx.QJ8x.Q8x.Q8 AT9x.x.xx.AKJ9xx J8xx.Txxx.9x.xxx
